https://nbviewer.jupyter.org/github/AdamSwenson/CanvasHacks/blob/master/Notebooks/Credit%20only%20assignment%20tools.ipynb?flush_cache=true

# Get started
Click the icon with 3 rings in the upper right (the tooltip says 'Execute on Binder') to create your own mini-server to run this notebook.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also  press shift+return to run it.

Try it with the next cell:

In [2]:
print("You've successfully run this cell!")

You've successfully run this cell!


You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( \[ * \]). Some tasks may take a couple of minutes to run, so it may appear frozen.

The next 2 cells will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/AdamSwenson/CanvasHacks)

In [3]:
# Install our library to the server
!pip install CanvasHacks

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
%cd ../

from CanvasHacks import environment
from CanvasHacks.RequestTools import get_all_course_assignments, get_assignments_needing_grading
from CanvasHacks.Configuration import InteractiveConfiguration
from CanvasHacks.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.DataManagement import DataStore
from CanvasHacks.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.UploadGradeTools import make_upload_button

/Users/adam/Dropbox/CanvasHacks
Reading credentials and settings from /Users/adam/Dropbox/CanvasHacks/private/canvas-credentials.ini


# Setup the configuration

Canvas provides a very nice API (a way to interacting with their server without going through the app or usual webpage) which we'll be using to get student submissions and submit grades. 

In the following, you'll give the program the credentials for accessing your student data. The program won't save your information and you'll have to re-enter it every time you use it. 

## Canvas API Token

You'll need to generate a token. This stands in for your username/password. 

Make sure you record the token in a secure place. It will not be saved in this notebook and you will have to re-enter it every time you use this tool.

https://canvas.instructure.com/doc/api/file.oauth.html#manual-token-generation


To manually generate a token for testing:

    Click the "profile" link in the top right menu bar, or navigate to /profile

    Under the "Approved Integrations" section, click the button to generate a new access token.
    
    Once the token is generated, you cannot view it again, and you'll have to generate a new token if you forget it. Remember that access tokens are password equivalent, so keep it secret.

Note that you don't have to click anything, just paste your token in the box and its value will be read when needed


In [ ]:
make_canvas_token_input()

## Course ids

You will also need the id number(s) of the courses you wish to grade. You can find them by logging into canvas and going to the course page. The number you are looking for will be in the url, immediately after '/courses/'

    For example, if the url is: https://canvas.csun.edu/courses/12345 
    
    The course number is: 12345
    
You will want to also write this number down somewhere secure since the values will be purged when you close the notebook.

Add the course ids one-at-a-time using the box and buttons below

In [ ]:
make_course_ids_input()

## Canvas url

Finally, we need the url of your campus canvas installation. When we make requests to the canvas server, we will be using this in our requests. For details, see the Canvas API documentation: https://canvas.instructure.com/doc/api/index.html

For CSUN users, it should be: 
    
    https://canvas.csun.edu/api/v1/courses

Just copy and paste the above into the box in the next cell.

For non-csun users, make sure you provide the equivalent up to the '/courses'

Just paste the url in the box, it will be read when needed

Do not add a slash at the end after '/courses'! 

In [ ]:
make_canvas_url_input()

# Figure out what needs grading

In [ ]:
print("These assignments need grading: ")
for sec in environment.CONFIG.course_ids:
    to_grade = get_assignments_needing_grading(sec)
    print(sec, to_grade)

# Download and process student submissions

Todo

    - Do we want to have more control over which assignments are graded? This just does all ungraded...

In [ ]:
results = []


for course_id in environment.CONFIG.course_ids:
    print('course', course_id)
    to_grade = get_assignments_needing_grading(course_id)
    
    for name, assignment_id in to_grade:
        store = DataStore(assignment_id=assignment_id, assignment_name=name, course_id=course_id)
        print("Processing {}".format(name))
        
        # download student submissions 
        response = PT.get_submissions(course_id, assignment_id)
        print("{} responses received".format(len(response)))
        store.submissions = PT.process_response_without_saving_files(response)
        
        # give preliminary credit for non-empty submissions
        c = GT.determine_credit(store.submissions)
        
        # stow the results in our data object
        store.credit = c['credit']
        store.no_credit = c['nocredit']
        store.print_counts()
        
        # Add the now full-of-data object to our results list
        results.append(store)

# Read student work and check that properly categorized

In [ ]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s)

# Upload the grades to canvas

In [ ]:
for store in results:
    make_upload_button(store)

# Final steps....

Go to Canvas and check your gradebook. There may be a couple of submissions which got skipped because the student uploaded a file in a format we couldn't extract the text from. 

(Out of 120 students, I usually find 0-2 that didn't get graded for this reason.)

Have a drink for a job well-done!

